In [ ]:
import pandas as pd
from lightautoml import AutoML
from lightautoml.tasks import Task
from sklearn.model_selection import train_test_split

# Пример данных
data = {
    'A': [1, 2, 3, 4, 5, 6],
    'B': [7, 8, 9, 10, 11, 12],
    'C': [13, 14, 15, 16, 17, 18],
    'target': [0, 1, 0, 1, 0, 1]
}

# Создание DataFrame
df = pd.DataFrame(data)

# Разделим данные на признаки (X) и целевую переменную (y)
X = df.drop(columns=['target'])
y = df['target']

# Разделим данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Определим задачу классификации
task = Task('binary')

# Настроим AutoML с указанием модели логистической регрессии и градиентных бустингов
automl = AutoML(task=task, 
                cpu_limit=2, 
                memory_limit=4096, 
                random_state=42,
                presets=['stacking'],  # Стекинг по умолчанию
                learner_params={
                    'model': ['LogisticRegression', 'LGBM', 'CatBoost', 'XGB'],  # Базовые модели
                    'final_model': 'LogisticRegression'  # метамодель
                },
                # Параметры для подбора гиперпараметров:
                hyperparams={
                    'LGBM': {
                        'num_leaves': [31, 50, 100],
                        'learning_rate': [0.01, 0.1, 0.2],
                        'n_estimators': [50, 100, 200]
                    },
                    'XGB': {
                        'max_depth': [3, 6, 10],
                        'learning_rate': [0.01, 0.1, 0.2],
                        'n_estimators': [50, 100, 200]
                    },
                    'CatBoost': {
                        'iterations': [100, 200, 500],
                        'learning_rate': [0.01, 0.1],
                        'depth': [3, 6, 10]
                    }
                })

# Обучаем модель
automl.fit(X_train, y_train)

# Оценка модели на тестовых данных
preds = automl.predict(X_test)

# Печать результатов
print("Предсказания:", preds)
